# Klasifikasi Koin Rupiah Menggunakan Machine Learning

## Topik
Klasifikasi nominal koin Rupiah (50, 100, 200, 500, 1000) berbasis citra digital menggunakan pendekatan Machine Learning klasik.

## Anggota Kelompok
1. Nama Anggota 1 – NIM
2. Nama Anggota 2 – NIM
3. Nama Anggota 3 – NIM

---

## Pendahuluan
Identifikasi nominal koin secara otomatis penting untuk sistem kasir, mesin sortir uang, dan aplikasi edukasi.  
Tantangan utama terletak pada kemiripan ukuran koin serta variasi pencahayaan dan latar belakang.

Penelitian ini menggunakan pendekatan **Machine Learning klasik** tanpa deep learning.


In [105]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

from skimage.feature import hog


## Paparan Data

Dataset berupa citra koin Rupiah hasil pengambilan kamera dengan variasi:
- Pencahayaan
- Sudut pengambilan
- Jumlah koin dalam satu gambar

### Kelas:
- 50
- 100
- 200
- 500
- 1000

### Sumber Dataset
Dataset dikumpulkan secara mandiri untuk keperluan akademik.


In [106]:
def extract_features(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (128, 128))

  # Otsu
  _, th = cv2.threshold(
    gray, 0, 255,
    cv2.THRESH_BINARY + cv2.THRESH_OTSU
  )

  # --- HOG ---
  hog_feat = hog(
    th,
    orientations=9,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2),
    block_norm='L2-Hys'
  )

  # --- Hu Moments (Shape Descriptor) ---
  # Hitung moment statistik
  moments = cv2.moments(th)
  # Hitung Hu Moments (7 nilai)
  hu_moments = cv2.HuMoments(moments)
  # Log-transformasi untuk stabilitas numerik
  hu_feat = -np.sign(hu_moments) * np.log(np.abs(hu_moments))
  hu_feat = hu_feat.flatten()

  # --- Global statistics ---
  white_ratio = np.sum(th == 255) / th.size
  mean_intensity = np.mean(gray)

  # --- CENTRAL-BOTTOM PATCH (ANGKA AREA) ---
  h, w = gray.shape
  patch = gray[int(h*0.55):int(h*0.9), int(w*0.25):int(w*0.75)]

  edges_patch = cv2.Canny(patch, 50, 150)
  edge_density = np.sum(edges_patch > 0) / edges_patch.size

  features = np.hstack([
    hog_feat,
    hu_feat,             # <--- FITUR BARU
    white_ratio,
    mean_intensity,
    edge_density
  ])

  return features, gray, th

In [107]:
X = []
y = []

DATASET_PATH = "dataset/train"
CLASSES = ["50", "100", "200", "500", "1000"]

for label in CLASSES:
  folder = os.path.join(DATASET_PATH, label)
  if not os.path.isdir(folder):
    continue

  for file in os.listdir(folder):
    img_path = os.path.join(folder, file)
    img = cv2.imread(img_path)
    if img is None:
      continue

    features, _, _ = extract_features(img)
    X.append(features)
    y.append(label)


X = np.array(X)
y = np.array(y)

print("Total data:", len(X))


Total data: 140


In [108]:
X_train, X_temp, y_train, y_temp = train_test_split(
  X, y, test_size=0.3, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
  X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Train:", len(X_train))
print("Validation:", len(X_val))
print("Test:", len(X_test))


Train: 98
Validation: 21
Test: 21


## Metode dan Eksperimen

### Metode
Model yang digunakan adalah **Support Vector Machine (SVM)** dengan kernel RBF.

### Alasan Pemilihan
- Cocok untuk dataset kecil–menengah
- Efektif pada data berdimensi tinggi (HOG)

### Metrik Evaluasi
- Accuracy
- Precision
- Recall
- F1-score
- Confusion Matrix


In [109]:
pipeline = Pipeline([
  ("scaler", StandardScaler()),
  ("svm", SVC())
])

param_grid = {
  "svm__C": [1, 10, 100],
  "svm__gamma": ["scale", 0.01, 0.001],
  "svm__kernel": ["rbf"]
}

grid = GridSearchCV(
  pipeline,
  param_grid,
  cv=3,
  scoring="accuracy",
  n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)


Best Params: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}


In [110]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

         100       1.00      0.40      0.57         5
        1000       0.50      1.00      0.67         4
         200       1.00      0.25      0.40         4
          50       0.50      1.00      0.67         4
         500       1.00      0.50      0.67         4

    accuracy                           0.62        21
   macro avg       0.80      0.63      0.59        21
weighted avg       0.81      0.62      0.59        21

[[2 2 0 1 0]
 [0 4 0 0 0]
 [0 1 1 2 0]
 [0 0 0 4 0]
 [0 1 0 1 2]]


In [ ]:
def detect_and_predict(image, model):
  output = image.copy()
  results = []

  # --- STEP 1: Preprocessing ---
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (9, 9), 0)

  # --- STEP 2: Otsu Threshold ---
  _, th = cv2.threshold(
    blur, 0, 255,
    cv2.THRESH_BINARY + cv2.THRESH_OTSU
  )

  if np.mean(th) > 127:
    th = cv2.bitwise_not(th)

  # --- STEP 3: Morfologi ---
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))
  th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)
  th = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)

  # --- STEP 4: Cari SEMUA contour koin ---
  contours, _ = cv2.findContours(
    th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
  )

  if len(contours) == 0:
    return output, []

  for c in contours:
    area = cv2.contourArea(c)
    if area < 5000:   # filter noise kecil
      continue

    (x, y), r = cv2.minEnclosingCircle(c)
    x, y, r = int(x), int(y), int(r)

    # --- ROI ---
    roi = image[y-r:y+r, x-r:x+r]
    if roi.size == 0:
      continue

    features, gray_roi, th_roi = extract_features(roi)
    pred = model.predict([features])[0]

    # --- Visualisasi ---
    cv2.circle(output, (x, y), r, (0, 255, 0), 3)
    cv2.putText(
      output,
      str(pred),
      (x - r, y - r - 10),
      cv2.FONT_HERSHEY_SIMPLEX,
      10,
      (0, 0, 255),   # MERAH
      10
    )

    results.append((gray_roi, th_roi))

  return output, results


In [112]:
import gradio as gr

def gradio_app(image):
  output, steps = detect_and_predict(image, best_model)

  gallery = []
  for gray, th in steps:
    gallery.append(gray)
    gallery.append(th)

  return output, gallery

iface = gr.Interface(
  fn=gradio_app,
  inputs=gr.Image(type="numpy", label="Upload Gambar Koin"),
  outputs=[
    gr.Image(label="Hasil Deteksi & Klasifikasi"),
    gr.Gallery(label="Preprocessing (Grayscale & Otsu)")
  ],
  title="Klasifikasi Koin Rupiah",
  description="Deteksi dan klasifikasi koin Rupiah menggunakan Otsu + HOG + SVM"
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
